In [1]:
import os, json, gzip 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet, stopwords

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [7]:
!python -m wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz


Saved under reviews_Electronics_5.json.gz


In [8]:
data = []
with gzip.open('reviews_Electronics_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
df = pd.DataFrame.from_dict(data)

In [22]:

# drop any rows w/ missing values
df = df.dropna()
# discover the actual counts
df.overall.value_counts()
# set sample size to labels w/ minimum count
sample_size = 50000
df_equal_overall = pd.DataFrame()
for i in df.overall.unique():
    X = df[df.overall == i].sample(sample_size)
    df_equal_overall = df_equal_overall.append(X)

C:\Users\HP\AppData\Local\Temp\ipykernel_7128\4117583541.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_equal_overall = df_equal_overall.append(X)
C:\Users\HP\AppData\Local\Temp\ipykernel_7128\4117583541.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_equal_overall = df_equal_overall.append(X)
C:\Users\HP\AppData\Local\Temp\ipykernel_7128\4117583541.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_equal_overall = df_equal_overall.append(X)
C:\Users\HP\AppData\Local\Temp\ipykernel_7128\4117583541.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_equal_overall = df_equal_overall.append(X)
C:\Users\HP\AppData\

In [16]:

stopwords_list = stopwords.words('english')

def ReviewProcessing(df):
    # remove non alphanumeric 
    df['review_cleaned'] = df.reviewText.str.replace('[^a-zA-Z0-9 ]', '', regex=True)
    # lowercase
    df.review_cleaned = df.review_cleaned.str.lower()
    # split into list
    df.review_cleaned = df.review_cleaned.str.split(' ')
    # remove stopwords
    df.review_cleaned = df.review_cleaned.apply(lambda x: [item for item in x if item not in stopwords_list])
    return df

In [33]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = nltk.stem.WordNetLemmatizer()

def get_lemmatize(sent):
    return " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sent)])


In [13]:
test_df = df_equal_overall.head(100)

In [14]:
print(test_df.head())

             reviewerID        asin            reviewerName  helpful  \
310830    AYF3LJ6AMXCEB  B000TD8XG6  Philip Itoh "Phil 808"  [3, 11]   
986115   A2TJ7RVA2HNM64  B004MMEI8W                   AFGUY   [0, 0]   
1272322  A1PNIXTVIDQZIF  B00752R4PK               Heather M   [0, 0]   
1266825  A15CV8QA5PIKEP  B00746W3HG     firefly2550 "stacy"   [0, 1]   
1678879   A774EQHWBJE4O  B00HPM1FZ0              Thomas Ray   [1, 1]   

                                                reviewText  overall  \
310830   I purchased these speakers because of the clea...      5.0   
986115   the Good:  I wanted over the ear headphones an...      5.0   
1272322  I don't use many of the features, but it plays...      5.0   
1266825  whats not to love about the ipad mini? i have ...      5.0   
1678879  This unit is a bit heavy but it works perfectl...      5.0   

                       summary  unixReviewTime   reviewTime  \
310830   7006 at a great price      1293753600  12 31, 2010   
986115        

In [29]:
clean_data = ReviewProcessing(df_equal_overall)

In [30]:
clean_data.review_cleaned = clean_data.review_cleaned.apply(' '.join)

In [35]:
test_df_clean = ReviewProcessing(test_df)
test_df.review_cleaned = test_df.review_cleaned.apply(' '.join)
test_df['review_cleaned_lemmatized'] = test_df.review_cleaned.apply(get_lemmatize)

In [36]:
clean_data['review_cleaned_lemmatized'] = clean_data.review_cleaned.apply(get_lemmatize)

In [18]:
from sklearn.naive_bayes import MultinomialNB

nb = Pipeline([('vectorize', CountVectorizer(ngram_range=(1, 2))),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [19]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier()),
               ])

In [20]:

from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(max_iter=500)),
               ])

In [21]:
x = clean_data['review_cleaned_lemmatized']
y = clean_data['overall']
X_train, X_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size=0.2, stratify=y, random_state = 44)


KeyError: 'review_cleaned_lemmatized'

In [ ]:
# Naive Bayes
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
print(accuracy_score(y_test, y_pred_nb))
print(confusion_matrix(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))

# SGD Classifier
sgd.fit(X_train, y_train)
y_pred_sgd = sgd.predict(X_test)
print(accuracy_score(y_test, y_pred_sgd))
print(confusion_matrix(y_test, y_pred_sgd))
print(classification_report(y_test, y_pred_sgd))

# Logistic Regression
logreg.fit(X_train, y_train)
y_pred_log = logreg.predict(X_test)
print(accuracy_score(y_test, y_pred_log))
print(confusion_matrix(y_test, y_pred_log))
print(classification_report(y_test, y_pred_log))

In [ ]:
from sklearn.model_selection import GridSearchCV

grid=[{'clf__solver': ['lbfgs', 'sag', 'saga'],
       'clf__C': [0.01, 0.1, 1]}]
lr = GridSearchCV(logreg, param_grid = grid, cv = 5, scoring='accuracy', verbose = 1, n_jobs = -1)
best_model = lr.fit(X_train, y_train)

print(best_model.best_estimator_)
print(best_model.best_score_)

y_pred_grid = best_model.predict(X_test)
print(confusion_matrix(y_test, y_pred_grid))
print(classification_report(y_test, y_pred_grid))
print(accuracy_score(y_test, y_pred_grid))